# Limpieza de un libro en texto plano (PLN)

Objetivo: cargar `data/libro.txt` y aplicar **normalización + lematización** (español) usando spaCy.


In [ ]:
from pathlib import Path
from collections import Counter

import spacy

ROOT = Path().resolve()
DATA_PATH = ROOT / "data" / "libro.txt"
OUTPUT_DIR = ROOT / "outputs"
SPACY_MODEL = "es_core_news_sm"

print("Proyecto:", ROOT)
print("Entrada:", DATA_PATH)
print("Salida:", OUTPUT_DIR)


## 1) Cargar texto

El archivo debe estar en **UTF-8**.

In [ ]:
text_raw = DATA_PATH.read_text(encoding="utf-8")
text_raw = " ".join(text_raw.split())

print("Chars:", len(text_raw))
print(text_raw[:400])


## 2) Cargar modelo spaCy (español)

Si el modelo no está instalado, se descarga automáticamente.

In [ ]:
try:
    nlp = spacy.load(SPACY_MODEL)
except OSError:
    from spacy.cli import download
    download(SPACY_MODEL)
    nlp = spacy.load(SPACY_MODEL)

doc = nlp(text_raw)
print("Tokens spaCy:", len(doc))
print([t.text for t in doc[:20]])


## 3) Limpieza (normalización + lematización)

Reglas:
- quitar stopwords
- quitar puntuación/espacios/números
- quedarse solo con tokens alfabéticos
- usar `token.lemma_` en minúsculas


In [ ]:
lemmas = []

for token in doc:
    if token.is_space or token.is_punct or token.like_num:
        continue
    if token.is_stop:
        continue
    if not token.is_alpha:
        continue

    lemma = token.lemma_.lower().strip()
    if lemma:
        lemmas.append(lemma)

print("Lemas:", len(lemmas))
print("Primeros 30:", lemmas[:30])


## 4) Guardar outputs

- `outputs/libro_lemmas.txt`
- `outputs/libro_normalizado.txt`
- `outputs/top_30_frecuencias.txt`


In [ ]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

(OUTPUT_DIR / "libro_lemmas.txt").write_text("\n".join(lemmas) + "\n", encoding="utf-8")
(OUTPUT_DIR / "libro_normalizado.txt").write_text(" ".join(lemmas) + "\n", encoding="utf-8")

freq = Counter(lemmas)
top_30 = freq.most_common(30)
(OUTPUT_DIR / "top_30_frecuencias.txt").write_text(
    "\n".join([f"{w}\t{c}" for w, c in top_30]) + "\n",
    encoding="utf-8",
)

print("Únicos:", len(freq))
print("Top 10:", top_30[:10])
